<a href="https://colab.research.google.com/github/meiruv/NIPS-Papers-Analysis/blob/main/NIPS_Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fetching the Data From Kaggle

In [1]:
! pip install -q kaggle

! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

! cp kaggle.json ~/.kaggle/
! kaggle datasets download -d rowhitswami/nips-papers-1987-2019-updated

 91% 97.0M/106M [00:01<00:00, 43.4MB/s]
100% 106M/106M [00:01<00:00, 59.6MB/s] 


In [2]:
import os
import zipfile

local_zip = 'nips-papers-1987-2019-updated.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('nips_papers')
zip_ref.close()

In [3]:
!ls nips_papers

authors.csv  papers.csv


# Pre Processing Papers

# LDA

## First Way - Via Gensim

In [ ]:
from gensim.corpora import Dictionary

texts = papers_df['full_text_clean']

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

import numpy
numpy.random.seed(1) # setting random seed to get the same results each time.

from gensim.models import ldamodel
model = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=10, minimum_probability=1e-8)
model.show_topics()

## Second Way - Using Scikit Learn

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_df=0.9, min_df=2, stop_words='english', token_pattern=r'(?u)\b[A-Za-z]+\b')
papers_tf_idf = tfidf.fit_transform(papers_df['full_text_clean'])

papers_tf_idf

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

LDA = LatentDirichletAllocation(n_components=15, random_state=101)

# This can take a while, we are dealing with large number of documents here
LDA.fit(papers_tf_idf)

In [ ]:
# What topics did we get?

for index, topic in enumerate(LDA.components_):
    print(f"THE TOP 15 WORDS FOR TOPIC #{index}")
    list_keywords = [tfidf.get_feature_names()[index] for index in topic.argsort()[-10:]]
    print(list_keywords)
    print("\n")